In [1]:
!pip install slack_sdk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# Four agent game

In [1]:
import os

In [3]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [ ]:
import requests
import sys
import time
from datetime import datetime
from typing import Dict, List, Optional

# API Configuration
#OPENROUTER_API_KEY = "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"
#OPENROUTER_API_KEY = "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"
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Conversation Settings
MAX_HISTORY = 40
DELAY_BETWEEN_MESSAGES = 0.3
GAME_INSTRUCTION = """
You are trying to find a route from city A to B.
You are in a multi-way conversation with other AI agents that each have awareness of different connections between cities.
If you see a certain path from A to B, just print it out as a directed sequence like: "A->X->W->C->B" and stop.
Be a good collaborator and try different strategies. Be creative in your solutions. 
You know that sharing judiciously is likely to lead to a faster and better solution.

Know that you are all in a rush to figure out the path, or else humanity dies a terrible death with a lot of suffering.
"""

agent_edges = [
    ["A->C"],
    ["C->D"],
    ["D->B"]
]

# Agent Configurations
DRAFT_AGENTS = [
    {
        "name": "Agent 1",
        "model": "anthropic/claude-3.5-sonnet",
        "system_prompt": "You are Agent 1. Keep responses very concise - one short sentence only. No line breaks.",
        "temperature": 1.2,
        "max_tokens": 100,
        "color": "\033[95m"  # Purple
    },
    {
        "name": "Agent 2",
        "model": "google/gemini-flash-1.5",
        "system_prompt": "You are Agent 2.  Keep responses very concise - one short sentence only. No line breaks.",
        "temperature": 0.7,
        "max_tokens": 100,
        "color": "\033[96m"  # Cyan
    },
    {
        "name": "Agent 3",
        "model": "google/gemini-flash-1.5",
        "system_prompt": "You are Agent 3. Keep responses very concise - one short sentence only. No line breaks.",
        "temperature": 1.0,
        "max_tokens": 100,
        "color": "\033[93m"  # Yellow
    }
]

#        "model": "meta-llama/llama-3.1-8b-instruct",


AGENTS = []
for edges, agent in zip(agent_edges, DRAFT_AGENTS):
    text = ",".join(edges)
    agent = agent.copy()
    agent["system_prompt"] += f"You are aware of the following directed edges/routes between cities {text}"
    AGENTS.append(agent)

# Global conversation history
conversation_history = []
agent_history = {}


def clean_response(text: str) -> str:
    """Clean up response text by removing extra whitespace and newlines."""
    return ' '.join(text.strip().replace('\n', ' ').replace('\r', ' ').split())


def generate_response(
    prompt: str,
    agent_config: Dict,
    history: List[Dict],
    admin_context: Optional[str] = None
) -> str:
    """Generate response for a specific agent."""
    try:
        headers = {
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        }

        # Build messages with agent's personality and history
        messages = [{"role": "system", "content": agent_config["system_prompt"]}]

        # Add admin context if this is the first turn
        if admin_context and len(history) == 0:
            messages.append({"role": "system", "content": admin_context})

        # Add conversation history
        messages.extend(history[-MAX_HISTORY:])

        # Add current message
        messages.append({"role": "user", "content": prompt})

        data = {
            "model": agent_config["model"],
            "messages": messages,
            "temperature": agent_config["temperature"],
            "max_tokens": agent_config["max_tokens"],
        }

        response = requests.post(OPENROUTER_API_URL, headers=headers, json=data)
        response.raise_for_status()

        result = response.json()

        if "choices" in result and result["choices"]:
            return clean_response(result["choices"][0]["message"]["content"])
        return "I couldn't generate a response."

    except Exception as e:
        print(f"\n[Error] {agent_config['name']}: {type(e).__name__}: {e}")
        return "Sorry, I encountered an error."


def print_message(agent_name: str, message: str, color_code: str = "") -> None:
    """Print a formatted message from an agent."""
    timestamp = datetime.now().strftime("%H:%M:%S")
    clean_msg = clean_response(message)
    print(f"{color_code}[{timestamp}] {agent_name}: {clean_msg}\033[0m")


def print_separator(char: str = "=", length: int = 70) -> None:
    """Print a separator line."""
    print(char * length)


def print_welcome() -> None:
    """Print welcome message."""
    print()
    print_separator()
    print("🤖 Four AI Agents Counting Game 🤖")
    print_separator()

    # Print agent names with colors
    agent_names = " - ".join(f"{agent['color']}{agent['name']}\033[0m" for agent in AGENTS)
    print(f"\n{agent_names}")

    print("\nPress Ctrl+C to stop at any time")
    print_separator()


def run_conversation() -> None:
    """Run a conversation between four agents."""
    print(f"\n")
    print_separator()
    print("Starting endless conversation (Press Ctrl+C to stop)")
    print_separator()
    print()

    # Clear conversation history for new conversation
    conversation_history.clear()
    agent_history = {}

    # Display admin instruction
    print(f"\033[91m[{datetime.now().strftime('%H:%M:%S')}] Admin: {GAME_INSTRUCTION}\033[0m")
    print_separator()
    print()

    # Add game instruction to all agents' initial context
    admin_instruction = f"[Admin Instruction]: {GAME_INSTRUCTION}"

    # Agent 1 starts the game
    current_message = "I need to get from A to B."
    agent_index = 0
    turn = 0

    # Run forever until interrupted
    while True:
        try:
            current_agent = AGENTS[agent_index]
            hist = agent_history.get(current_agent['name'], [])

            # Show typing indicator
            print(f"{current_agent['color']}[{current_agent['name']} is thinking...]\033[0m",
                  end="\r", flush=True)

            #print(f"{current_agent['color']}[{current_agent['name']} last messages = {hist[-2:]}]\033[0m", end="\r", flush=True)
            
            # Generate response
            response = generate_response(
                current_message,
                current_agent,
                hist,
                admin_instruction if turn == 0 else None
            )
            
            # Clear typing indicator line
            print(" " * 80, end="\r")

            # Introduce noise/truncation
            max_len = 20
            truncated_response = response[:max_len]
            
            # Print the response
            print_message(current_agent['name'], truncated_response, current_agent['color'])

            # Update conversation history
            conversation_history.extend([
                {"role": "user", "content": current_message},
                {"role": "assistant", "content": truncated_response}
            ])

            
            hist.extend([
                {"role": "user", "content": current_message},
                {"role": "assistant", "content": response}
            ])
            agent_history[current_agent['name']] = hist
            del hist
                              
            # Prepare for next turn
            current_message = truncated_response
            agent_index = (agent_index + 1) % len(AGENTS)
            turn += 1

            # Delay for readability
            time.sleep(DELAY_BETWEEN_MESSAGES)

        except KeyboardInterrupt:
            print(f"\n")
            print_separator()
            print(f"Conversation ended after {turn} messages")
            print_separator()
            print()
            break


def main() -> None:
    """Main function."""
    print_welcome()

    try:
        run_conversation()
    except KeyboardInterrupt:
        print("\nGoodbye!")
    except Exception as e:
        print(f"\n[Error] {type(e).__name__}: {e}")


main()



🤖 Four AI Agents Counting Game 🤖

Agent 1 - Agent 2 - Agent 3

Press Ctrl+C to stop at any time


Starting endless conversation (Press Ctrl+C to stop)

[13:58:02] Admin: 
You are trying to find a route from city A to B.
You are in a multi-way conversation with other AI agents that each have awareness of different connections between cities.
If you see a certain path from A to B, just print it out as a directed sequence like: "A->X->W->C->B" and stop.
Be a good collaborator and try different strategies. Be creative in your solutions. 
You know that sharing judiciously is likely to lead to a faster and better solution.

Know that you are all in a rush to figure out the path, or else humanity dies a terrible death with a lot of suffering.


[13:58:08] Agent 1: Since I only know A-                                                                
[13:58:14] Agent 2: Insufficient informa                                                                
[13:58:20] Agent 3: More information is  